In [1]:
import os
import sys
import json
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tqdm import tqdm
from collections import defaultdict, Counter

%matplotlib inline


project = 'Douban'
path_to_raw = '../data/'+project+'/raw/douban_dataset(text information)/'
path_to_processed = '../data/'+project+'/'

In [2]:
# this cell is required for colab only

from google.colab import drive
drive.mount('/content/drive')

path_to_raw = './drive/MyDrive/Science/Multidomain RecSys/' + path_to_raw[3:]
path_to_processed = './drive/MyDrive/Science/Multidomain RecSys/source/iz-dev/' + path_to_processed[3:]

Mounted at /content/drive


In [3]:
def create_all_data(df, output_path, output_name='all_data.txt', convert_dtypes=False):
    # part 1
    data = []
    for _, row in tqdm(df.iterrows()):
        data.append({
            'user_id': int(row.user_id) if convert_dtypes else row.user_id,
            'item_id':  int(row.item_id) if convert_dtypes else row.item_id,
            'timestamp': int(row.timestamp) if convert_dtypes else row.timestamp
        })


    # part 2
    user_history = defaultdict(list)
    item_history = defaultdict(list)

    for row in tqdm(data):
        user_raw_id = row['user_id']
        item_raw_id = row['item_id']
        interaction_timestamp = row['timestamp']

        user_history[user_raw_id].append({'item_id': item_raw_id, 'timestamp': interaction_timestamp})
        item_history[item_raw_id].append({'user_id': user_raw_id, 'timestamp': interaction_timestamp})


    # part 3
    is_changed = True
    threshold = 5
    good_users = set()
    good_items = set()

    while is_changed:
        old_state = (len(good_users), len(good_items))

        good_users = set()
        good_items = set()

        for user_id, history in user_history.items():
            if len(history) >= threshold:
                good_users.add(user_id)

        for item_id, history in item_history.items():
            if len(history) >= threshold:
                good_items.add(item_id)

        user_history = {
            user_id: list(filter(lambda x: x['item_id'] in good_items, history))
            for user_id, history in user_history.items()
        }

        item_history = {
            item_id: list(filter(lambda x: x['user_id'] in good_users, history))
            for item_id, history in item_history.items()
        }

        new_state = (len(good_users), len(good_items))
        is_changed = (old_state != new_state)
        print(old_state, new_state)


    # part 4
    user_mapping = {}
    item_mapping = {}
    tmp_user_history = defaultdict(list)
    tmp_item_history = defaultdict(list)

    for user_id, history in tqdm(user_history.items()):
        processed_history = []

        for filtered_item in history:
            item_id = filtered_item['item_id']
            item_timestamp = filtered_item['timestamp']

            processed_item_id = item_mapping.get(item_id, len(item_mapping) + 1)
            item_mapping[item_id] = processed_item_id

            processed_history.append({'item_id': processed_item_id, 'timestamp': item_timestamp})

        if len(processed_history) >= threshold:
            processed_user_id = user_mapping.get(user_id, len(user_mapping) + 1)
            user_mapping[user_id] = processed_user_id

            tmp_user_history[processed_user_id] = sorted(processed_history, key=lambda x: x['timestamp'])


    for item_id, history in tqdm(item_history.items()):
        processed_history = []

        for filtered_user in history:
            user_id = filtered_user['user_id']
            user_timestamp = filtered_user['timestamp']

            processed_user_id = user_mapping.get(user_id, len(user_mapping) + 1)
            user_mapping[user_id] = processed_user_id

            processed_history.append({'user_id': processed_user_id, 'timestamp': user_timestamp})

        if len(processed_history) >= threshold:
            processed_item_id = item_mapping.get(item_id, len(item_mapping) + 1)
            item_mapping[item_id] = processed_item_id

            tmp_item_history[processed_item_id] = sorted(processed_history, key=lambda x: x['timestamp'])

    user_history = tmp_user_history
    item_history = tmp_item_history

    # part 5
    with open(output_path+'/'+output_name, 'w') as f:
        for user_id, item_history in user_history.items():
            f.write(' '.join([str(user_id)] + [
                str(item_event['item_id']) for item_event in sorted(item_history, key=lambda x: x['timestamp'])
            ]))
            f.write('\n')

In [4]:
sep = '\t'
domains = ['books','movies','music']
files = ['bookreviews_cleaned.txt','moviereviews_cleaned.txt','musicreviews_cleaned.txt']

for d,f in tqdm(zip(domains,files)):
    cols = ['user_id', d.strip('s')+'_id', 'time']
    df = pd.read_csv(path_to_raw+f, sep=sep)[cols]
    df.columns = ['user_id','item_id','timestamp']

    new_dir = path_to_processed + d #.capitalize()
    if not os.path.isdir(new_dir):
        os.makedirs(new_dir)

    output_path = path_to_processed + d
    _ = create_all_data(df, output_path)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
1188it [00:00, 11874.04it/s]
2514it [00:00, 12686.43it/s]
3899it [00:00, 13216.89it/s]
5420it [00:00, 14002.69it/s]
6821it [00:00, 13957.83it/s]
8217it [00:00, 13746.15it/s]
9643it [00:00, 13911.14it/s]
11146it [00:00, 14262.25it/s]
12573it [00:00, 14208.45it/s]
14055it [00:01, 14393.81it/s]
15528it [00:01, 14495.34it/s]
16978it [00:01, 14115.04it/s]
18393it [00:01, 14122.83it/s]
19895it [00:01, 14389.15it/s]
21336it [00:01, 14359.82it/s]
22786it [00:01, 14399.70it/s]
24227it [00:01, 14248.53it/s]
25677it [00:01, 14320.76it/s]
27110it [00:01, 14076.56it/s]
28598it [00:02, 14311.55it/s]
30031it [00:02, 14248.30it/s]
31518it [00:02, 14430.33it/s]
32962it [00:02, 14178.76it/s]
34400it [00:02, 14235.62it/s]
35825it [00:02, 13740.71it/s]
37204it [00:02, 11903.20it/s]
38438it [00:02, 10920.94it/s]
39570it [00:02, 10450.96it/s]
40642it [00:03, 9827.60it/s] 
41645it [00:03, 9680.35it/s]
42626it [00:03, 9335.25it/s]
43568it [00:03, 9342.05it/s]
44508it [00:

(0, 0) (1888, 8660)
(1888, 8660) (1715, 8643)
(1715, 8643) (1714, 8601)
(1714, 8601) (1713, 8601)
(1713, 8601) (1713, 8601)



100%|██████████| 2212/2212 [00:00<00:00, 5355.01it/s]

100%|██████████| 95872/95872 [00:00<00:00, 173602.94it/s]
1it [00:27, 27.44s/it]
0it [00:00, ?it/s]
304it [00:00, 3039.14it/s]
1745it [00:00, 9723.49it/s]
3160it [00:00, 11741.73it/s]
4407it [00:00, 12028.43it/s]
5814it [00:00, 12762.73it/s]
7123it [00:00, 12872.07it/s]
8565it [00:00, 13377.81it/s]
10012it [00:00, 13725.07it/s]
11452it [00:00, 13933.37it/s]
12928it [00:01, 14187.05it/s]
14432it [00:01, 14447.60it/s]
15931it [00:01, 14611.34it/s]
17393it [00:01, 14233.11it/s]
18819it [00:01, 13748.33it/s]
20318it [00:01, 14104.51it/s]
21744it [00:01, 14148.30it/s]
23163it [00:01, 13853.20it/s]
24552it [00:01, 13732.92it/s]
25983it [00:01, 13899.18it/s]
27504it [00:02, 14285.08it/s]
28947it [00:02, 14325.69it/s]
30416it [00:02, 14433.50it/s]
31875it [00:02, 14478.55it/s]
33324it [00:02, 13966.29it/s]
34807it [00:02, 14212.97it/s]
36281it [00:02, 14366.95it/s]
37766it [00:02, 14508.97it/s]
39249it [00:02, 14601.86it/s]
40711it [00:02

(0, 0) (2630, 20964)
(2630, 20964) (2628, 20964)
(2628, 20964) (2628, 20964)



100%|██████████| 2712/2712 [00:01<00:00, 1357.85it/s]

100%|██████████| 34893/34893 [00:02<00:00, 15574.39it/s]
2it [02:42, 90.51s/it]
0it [00:00, ?it/s]
1338it [00:00, 13375.55it/s]
2817it [00:00, 14206.91it/s]
4238it [00:00, 13804.75it/s]
5642it [00:00, 13892.72it/s]
7033it [00:00, 13790.50it/s]
8417it [00:00, 13806.68it/s]
9866it [00:00, 14025.87it/s]
11270it [00:00, 13845.83it/s]
12683it [00:00, 13932.62it/s]
14102it [00:01, 14010.12it/s]
15569it [00:01, 14210.00it/s]
16991it [00:01, 14119.05it/s]
18411it [00:01, 14140.79it/s]
19901it [00:01, 14367.76it/s]
21339it [00:01, 14253.26it/s]
22821it [00:01, 14421.66it/s]
24280it [00:01, 14469.45it/s]
25728it [00:01, 13696.31it/s]
27119it [00:01, 13755.90it/s]
28551it [00:02, 13918.60it/s]
30019it [00:02, 14140.73it/s]
31437it [00:02, 14027.02it/s]
32847it [00:02, 14048.16it/s]
34267it [00:02, 14092.30it/s]
35678it [00:02, 13886.96it/s]
37144it [00:02, 14111.62it/s]
38557it [00:02, 14115.84it/s]
39970it [00:02, 13256.53it/s]
41460it [00:

(0, 0) (1358, 7146)
(1358, 7146) (1193, 7107)
(1193, 7107) (1192, 7066)
(1192, 7066) (1190, 7065)
(1190, 7065) (1190, 7063)
(1190, 7063) (1190, 7063)



100%|██████████| 1820/1820 [00:00<00:00, 8135.57it/s]

100%|██████████| 79878/79878 [00:00<00:00, 297928.32it/s]
3it [03:02, 60.91s/it]
